<a href="https://colab.research.google.com/github/JoannaLe/cancer-detection/blob/master/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, Model

import numpy as np
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [1]:
# Open Google Cloud Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [16]:
print(os.getcwd())
os.chdir("/content")
!ls

/
gdrive	sample_data


In [25]:
# get X training data
x_path = "gdrive/My Drive/cancer-detection/patch-zoom-4"
print(os.listdir(x_path))
X_train = []

for patch_file in os.listdir(x_path):
  patch_arr = np.loadtxt(patch_file, dtype=int)
  X_train.append(patch_arr)
  
print(type(X_train[0]))
print(len(X_train))

# get Y training data
y_path = "gdrive/My Drive/cancer-detection/mask-zoom-4"
print(os.listdir(y_path))
y_train = []

for mask_file in os.listdir(x_path):
  mask_path = os.path.join(y_path, mask_file)
  mask = open(mask_path, 'r').read()
  y_train.append(mask)

['patch_0', 'patch_1', 'patch_2', 'patch_3', 'patch_4', 'patch_5', 'patch_6', 'patch_7', 'patch_8', 'patch_10', 'patch_9', 'patch_11']
/content
[
12


## Model 1
4 resnet models with a concatenation layer

In [0]:
img_shape = (300,300,3)

model1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model2 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model3 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model4 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)

# concatenation layer
concat = Average()([model1.output, model2.output, model3.output, model4.output])

full_model = Model([model1.input, model2.input, model3.input, model4.input], concat)


In [0]:
# to train, call
EPOCHS = 20
BATCH_SIZE = 32

full_model.fit([X_train1, X_train2, X_train3, X_train4],
               [Y_train1, Y_train2, Y_train3, Y_train4],
                epochs=EPOCHS,
                batch_size=BATCH_SIZE)

score = model.evaluate([X_test1, X_test2, X_test3, X_test4],
                       [Y_test1, Y_test2, Y_test3, Y_test4],
                       batch_size=BATCH_SIZE)

# Model 2
Transfer Learning using weights from Resnet \\
Extract features from Resnet weights and training/test data

In [0]:
img_shape = (300,300,3)

base = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
